In [5]:
# import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.svm import SVC 

In [6]:
# read in file, view
df = pd.read_csv("Merged_V3.csv")
df.head()

,Unnamed: 0,Total Population,ZIP Code,Median Income,Has?
0,0,1072,1001,31603,0
1,1,2673,1002,14855,0
2,2,1528,1003,3973,0
3,3,310,1005,33312,0
4,4,821,1007,40089,0


In [17]:
# drop non-features, view
df2 = df.drop("ZIP Code", axis=1)
df2 = df2.drop("Has?", axis=1)
df2.head()


,Unnamed: 0,Total Population,Median Income
0,0,1072,31603
1,1,2673,14855
2,2,1528,3973
3,3,310,33312
4,4,821,40089


In [15]:
df2 =df2.rename(index=str, columns={"Total Population":"Total_Population", "Median Income":"Median_Income"})

In [18]:
# Create feature matrix
X = df2.Total_Population

# Create target vector
y = df2.Labels

# Remove first 40 observations
X = X[40:] #X[40:,:] if more than one feature
y = y[40:]

# Create binary target vector indicating if class 0
y = np.where((y == 0), 0, 1)

# Look at the imbalanced target vector
y[:200]

AttributeError: 'DataFrame' object has no attribute 'Total_Population'

In [13]:
# Indicies of each class' observations
i_class0 = np.where(y == 0)[0]
i_class1 = np.where(y == 1)[0]

# Number of observations in each class
n_class0 = len(i_class0)
n_class1 = len(i_class1)

# For every observation of class 1, randomly sample from class 0 without replacement
i_class0_downsampled = np.random.choice(i_class0, size=n_class1, replace=False)

# Join together class 1's target vector with the downsampled class 0's target vector
NewVector = np.hstack((y[i_class1], y[i_class0_downsampled]))

In [14]:
vector1 = pd.DataFrame(i_class1)
vector1 = vector1.rename(columns={ vector1.columns[0]: "Total_Population" })
vector1['class'] = 1
vector1.head()

,Total_Population,class
0,102,1
1,106,1
2,113,1
3,118,1
4,146,1


In [15]:
vector0 = pd.DataFrame(i_class0_downsampled)
vector0 = vector0.rename(columns={ vector0.columns[0]: "Total_Population" })
vector0['class'] = 0
vector0.head()

,Total_Population,class
0,17244,0
1,32495,0
2,17111,0
3,22686,0
4,19467,0


In [16]:
df_downsample = pd.concat([vector1, vector0])
df_downsample

,Total_Population,class
0,102,1
1,106,1
2,113,1
3,118,1
4,146,1
5,279,1
6,290,1
7,306,1
8,312,1
9,322,1


In [17]:
# Assign X (data) and y (target)
X = df_downsample.drop("class", axis=1)
y = df_downsample["class"]
print(X.shape, y.shape)

(4334, 1) (4334,)


In [18]:
# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [21]:
# Support vector machine linear classifier
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [22]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.489


In [23]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.43      0.18      0.25       525
           1       0.50      0.78      0.61       559

   micro avg       0.49      0.49      0.49      1084
   macro avg       0.47      0.48      0.43      1084
weighted avg       0.47      0.49      0.44      1084



In [24]:
# DF with non-CMG locations
Non_CMG_Zips = df[df.Labels != 1]
Non_CMG_Zips2 = Non_CMG_Zips.drop("Count", axis=1)
Non_CMG_Zips3 = Non_CMG_Zips2.drop("Zip_Code", axis=1)
X2 = Non_CMG_Zips3.drop("Labels", axis=1)
X2.head()

,Total_Population
0,17599
1,39209
2,50135
3,6304
4,27590


In [28]:
predictions2 = model.predict(X2)
A = pd.DataFrame(predictions2)
A.sum()

0    9796
dtype: int64

In [29]:
Non_CMG_Zips2["Prediction"] = predictions2
Non_CMG_Zips2

,Labels,Zip_Code,Total_Population,Prediction
0,0,601,17599,1
1,0,602,39209,1
2,0,603,50135,1
3,0,606,6304,0
4,0,610,27590,1
5,0,612,62566,1
6,0,616,10687,1
7,0,617,24508,1
8,0,622,7405,1
9,0,623,42321,1
